In [ ]:
#Importing the libraries 
import pandas as pd

import json

import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from string import punctuation
from nltk.corpus import stopwords
from collections import defaultdict
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn import model_selection, naive_bayes, svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tabulate import tabulate

!pip install ibm-watson
!pip install ibm-cloud-sdk-core
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1  import Features, EmotionOptions,SentimentOptions

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
# Reading the Dataset
df = pd.read_csv('reddit_v3S.csv')
df['usertext']=df['usertext'].astype('str')
df['label']=df['label'].astype('int')

In [ ]:
# Getting basic scores such as sentiment score & emotions scores (Anger, Disgust, Sadness, Fear, Joy)
def getNLUFeatures(post):

  post_length = post.count(' ')

  watson_li = []

  if (post_length >= 3):
    response = service.analyze(
      text=post,
      language = 'en',
      features=Features(
          sentiment=SentimentOptions(),
          emotion=EmotionOptions(),
      )).get_result()
    #get_esult() gives a py object, 
    x = json.dumps(response, indent=2)
    #obtaining a py dict
    y = json.loads(x)
  
    #overall post sentiment score
    watson_li.append(y['sentiment']['document']['score'])
    
    #overall post emotion score
    val_anger = y['emotion']['document']['emotion']['anger']
    val_sadness = y['emotion']['document']['emotion']['sadness']
    val_disgust = y['emotion']['document']['emotion']['disgust']
    val_joy = y['emotion']['document']['emotion']['joy']
    val_fear = y['emotion']['document']['emotion']['fear']
    
    watson_li.extend([val_anger,val_sadness,val_disgust,val_joy,val_fear])

  else:
    watson_li.extend([0,0,0,0,0,0])
  "text": "apples",
  watson_df = pd.DataFrame(watson_li).transpose()
  return watson_df

X_featuresNLU = pd.DataFrame()
for post in df['usertext']:
  post_NLU = getNLUFeatures(post)
  X_featuresNLU = X_featuresNLU.append(post_NLU, ignore_index=True)

X_featuresNLU.head()

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

,0,1,2,3,4,5
0,-0.906167,0.272111,0.751128,0.090044,0.012919,0.174592
1,-0.813771,0.133655,0.698303,0.512980,0.040216,0.713078
2,-0.621066,0.167296,0.684554,0.041046,0.025701,0.374354
3,-0.724783,0.126606,0.715237,0.117129,0.006873,0.350638
4,-0.389862,0.849189,0.247317,0.046398,0.036702,0.020026


In [ ]:
# Reading NLU Features from Drive
X_featuresNLU = pd.read_csv('watsonNLU.csv')
X_featuresNLU.head()

,sentiment,anger,sadness,disgust,joy,fear
0,-0.906167,0.272111,0.751128,0.090044,0.012919,0.174592
1,-0.813771,0.133655,0.698303,0.512980,0.040216,0.713078
2,-0.621066,0.167296,0.684554,0.041046,0.025701,0.374354
3,-0.724783,0.126606,0.715237,0.117129,0.006873,0.350638
4,-0.389862,0.849189,0.247317,0.046398,0.036702,0.020026


In [ ]:
# Preprocessing to apply TFIDF
def preprocessPost(post):
  post = post.lower() # convert text to lower-case
  post = post.strip() # removed leading and trailing whitespace
  post = post.encode('ascii', 'ignore').decode('ascii') #remove emojis
  post = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', post) #removed URLs
  post = re.sub(r'\d+','', post) #removed digits
  post = re.sub(r'#([^\s]+)', r'\1', post) # remove hashtag

  # Replacing  short words
  post = re.sub(r"n\'t", " not", post)
  post = re.sub(r"\'re", " are", post)
  post = re.sub(r"\'s", " is", post)
  post = re.sub(r"\'d", " would", post)
  post = re.sub(r"\'ll", " will", post)
  post = re.sub(r"\'t", " not", post)
  post = re.sub(r"\'ve", " have", post)
  post = re.sub(r"\'m", " am", post)
  post = "".join([c for c in post if c not in string.punctuation])
  post = word_tokenize(post)

  #important stopwords that shouldn't be removed from Choudhary's study.
  ImpStopWords = ['to','i','you','he','she','they','her','we','my','myself','me','no',]
  MyStopWords = []

  for word in stopwords.words('english'):
    if word not in ImpStopWords:
      MyStopWords.append(word)

  FinalWords = []
  
  lemmatizer = WordNetLemmatizer()

  for word, tag in pos_tag(post):
       if word not in MyStopWords and word.isalpha():
        word_Final = lemmatizer.lemmatize(word)
        FinalWords.append(word_Final)

  post = str(FinalWords)

  return post

print("Preprocessing Posts....")
df['usertext'] = df['usertext'].apply(lambda x: preprocessPost(x))
df['usertext'].head()

Preprocessing Posts....


0    ['i', 'want', 'to', 'end', 'my', 'life', 'badl...
1    ['my', 'relationship', 'complicated', 'painful...
2    ['i', 'owe', 'lot', 'money', 'i', 'to', 'workt...
3    ['october', 'i', 'overdosed', 'i', 'felt', 'al...
4    ['everyone', 'tell', 'me', 'wonderful', 'i', '...
Name: usertext, dtype: object

In [ ]:
print("Applying TFIDF Features....")
Tfidf_vect = TfidfVectorizer(ngram_range=(2,3),max_features=100) #so that it takes 2-3 words combined
X_tfidf = Tfidf_vect.fit_transform(df['usertext'].values.astype('U'))
X_tfidf = pd.DataFrame(X_tfidf.todense())
X_tfidf.head()

Applying TFIDF Features....


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,0.357835,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.593627,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.438831,0.0,0.0,0.0,0.0,0.0000,0.0,0.439568,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.715128,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.253029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.2423,0.0,0.482339,0.0,0.0,0.24824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.885325,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.464973,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.724869,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.625759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Combining Watson NLU Features & TFIDF Features
features_df = pd.concat([X_featuresNLU,X_tfidf],axis=1)
final_df = pd.concat([features_df,df['label']],axis=1)
final_df.head()

,sentiment,anger,sadness,disgust,joy,fear,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,label
0,-0.906167,0.272111,0.751128,0.090044,0.012919,0.174592,0.357835,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.438831,0.0,0.0,0.0,0.0,0.0000,0.0,0.439568,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,-0.813771,0.133655,0.698303,0.512980,0.040216,0.713078,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.715128,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.253029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.2423,0.0,0.482339,0.0,0.0,0.24824,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,-0.621066,0.167296,0.684554,0.041046,0.025701,0.374354,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.464973,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,-0.724783,0.126606,0.715237,0.117129,0.006873,0.350638,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,-0.389862,0.849189,0.247317,0.046398,0.036702,0.020026,0.000000,0.0,0.0,0.0,0.0,0.0,0.625759,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
X = final_df[final_df.columns[:-1]]
Y = final_df['label']
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(X,Y,test_size=0.2, random_state=2000)

In [ ]:
# Classifier - Naive Bayes
scaler = MinMaxScaler()
scaled_TrainX = scaler.fit_transform(Train_X)
scaled_TestX = scaler.fit_transform(Test_X)
Naive = naive_bayes.MultinomialNB()
Naive.fit(scaled_TrainX,Train_Y)
predictions_NB = Naive.predict(scaled_TestX)

print("MNB Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
print("MNB Precision Score -> ",precision_score(predictions_NB, Test_Y)*100)
print("MNB Recall Score -> ",recall_score(predictions_NB, Test_Y)*100)
print("MNB F1 Score -> ",f1_score(predictions_NB, Test_Y)*100)

MNB Accuracy Score ->  84.0
MNB Precision Score ->  90.0
MNB Recall Score ->  80.35714285714286
MNB F1 Score ->  84.90566037735849


In [ ]:
#Logistic Regression
LR = LogisticRegression(solver='liblinear', random_state=2000)
LR.fit(Train_X,Train_Y)
predictions_LR = LR.predict(Test_X)

print("LR Accuracy Score -> ",accuracy_score(predictions_LR, Test_Y)*100)
print("LR Precision Score -> ",precision_score(predictions_LR, Test_Y)*100)
print("LR Recall Score -> ",recall_score(predictions_LR, Test_Y)*100)
print("LR F1 Score -> ",f1_score(predictions_LR, Test_Y)*100)

LR Accuracy Score ->  87.0
LR Precision Score ->  90.0
LR Recall Score ->  84.90566037735849
LR F1 Score ->  87.37864077669903


In [ ]:
#SVM
SVM = svm.SVC(C=1.0, kernel='linear', degree=1, gamma='auto')
SVM.fit(Train_X,Train_Y)
predictions_SVM = SVM.predict(Test_X)

print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM Precision Score -> ",precision_score(predictions_SVM, Test_Y)*100)
print("SVM Recall Score -> ",recall_score(predictions_SVM, Test_Y)*100)
print("SVM F1 Score -> ",f1_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  88.0
SVM Precision Score ->  90.0
SVM Recall Score ->  86.53846153846155
SVM F1 Score ->  88.23529411764707


In [ ]:
#Classifier - Random Forest
RF = RandomForestClassifier(n_estimators=100)
RF.fit(Train_X,Train_Y)
predictions_RF = RF.predict(Test_X)

print("RF Accuracy Score -> ",accuracy_score(predictions_RF, Test_Y)*100)
print("RF Precision Score -> ",precision_score(predictions_RF, Test_Y)*100)
print("RF Recall Score -> ",recall_score(predictions_RF, Test_Y)*100)
print("RF F1 Score -> ",f1_score(predictions_RF, Test_Y)*100)

RF Accuracy Score ->  91.0
RF Precision Score ->  94.0
RF Recall Score ->  88.67924528301887
RF F1 Score ->  91.2621359223301


In [ ]:
#Classifier - XGBoost
params = {
    'objective': 'binary:logistic',
    'max_depth': 2,
    'learning_rate': 1,
    'silent': 1,
    'n_estimators': 5
}

XGB = XGBClassifier(**params).fit(Train_X,Train_Y)
predictions_XGB = XGB.predict(Test_X)

print("XGB Accuracy Score -> ",accuracy_score(predictions_XGB, Test_Y)*100)
print("XGB Precision Score -> ",precision_score(predictions_XGB, Test_Y)*100)
print("XGB Recall Score -> ",recall_score(predictions_XGB, Test_Y)*100)
print("XGB F1 Score -> ",f1_score(predictions_XGB, Test_Y)*100)

XGB Accuracy Score ->  88.0
XGB Precision Score ->  86.0
XGB Recall Score ->  89.58333333333334
XGB F1 Score ->  87.75510204081633


In [ ]:
# 10-Fold Cross Validation

avg_result, h = [], ['Model', 'Accuracy', 'Precision', 'Recall', 'F1',]
mnb_acc, mnb_pre, mnb_rec, mnb_f1 = [], [], [], []
lr_acc, lr_pre, lr_rec, lr_f1 = [], [], [], []
svm_acc, svm_pre, svm_rec, svm_f1 = [], [], [], []
rf_acc, rf_pre, rf_rec, rf_f1 = [], [], [], []
xgb_acc, xgb_pre, xgb_rec, xgb_f1 = [], [], [], []

X = final_df[final_df.columns[:-1]]
y = final_df['label']
num_fold = 10
kf = KFold(n_splits=num_fold, shuffle=True, random_state=0)

for train_index, test_index in kf.split(X):
  num_fold -= 1
  Train_X, Test_X = X.iloc[train_index], X.iloc[test_index]
  Train_Y, Test_Y = y.iloc[train_index], y.iloc[test_index]
  
  # Naive Bayes
  scaler = MinMaxScaler()
  scaled_TrainX = scaler.fit_transform(X_train)
  scaled_TestX = scaler.fit_transform(Test_X)
  Naive = naive_bayes.MultinomialNB() #add parameter
  Naive.fit(scaled_TrainX,Train_Y)
  predictions_NB = Naive.predict(scaled_TestX)
  mnb_acc.append(accuracy_score(predictions_NB, Test_Y)*100)
  mnb_pre.append(precision_score(predictions_NB, Test_Y)*100)
  mnb_rec.append(recall_score(predictions_NB, Test_Y)*100)
  mnb_f1.append(f1_score(predictions_NB, Test_Y)*100)

  # Logistic Regression
  LR = LogisticRegression(solver='liblinear', random_state=2000)
  LR.fit(Train_X,Train_Y)
  predictions_LR = LR.predict(Test_X)
  lr_acc.append(accuracy_score(predictions_LR, Test_Y)*100)
  lr_pre.append(precision_score(predictions_LR, Test_Y)*100)
  lr_rec.append(recall_score(predictions_LR, Test_Y)*100)
  lr_f1.append(f1_score(predictions_LR, Test_Y)*100)

  # SVM
  SVM = svm.SVC(C=1.0, kernel='linear', degree=1, gamma='auto')
  SVM.fit(Train_X,Train_Y)
  predictions_SVM = SVM.predict(Test_X)
  svm_acc.append(accuracy_score(predictions_SVM, Test_Y)*100)
  svm_pre.append(precision_score(predictions_SVM, Test_Y)*100)
  svm_rec.append(recall_score(predictions_SVM, Test_Y)*100)
  svm_f1.append(f1_score(predictions_SVM, Test_Y)*100)

  # Random Forest
  RF = RandomForestClassifier(n_estimators=100)
  RF.fit(Train_X,Train_Y)
  predictions_RF = RF.predict(Test_X)
  rf_acc.append(accuracy_score(predictions_RF, Test_Y)*100)
  rf_pre.append(precision_score(predictions_RF, Test_Y)*100)
  rf_rec.append(recall_score(predictions_RF, Test_Y)*100)
  rf_f1.append(f1_score(predictions_RF, Test_Y)*100)

  # XGBoost
  params = {
      'objective': 'binary:logistic',
      'max_depth': 2,
      'learning_rate': 1,
      'silent': 1,
      'n_estimators': 5
  }
  XGB = XGBClassifier(**params).fit(Train_X,Train_Y)
  predictions_XGB = XGB.predict(Test_X)

  xgb_acc.append(accuracy_score(predictions_XGB, Test_Y)*100)
  xgb_pre.append(precision_score(predictions_XGB, Test_Y)*100)
  xgb_rec.append(recall_score(predictions_XGB, Test_Y)*100)
  xgb_f1.append(f1_score(predictions_XGB, Test_Y)*100)

# 10-Fold Cross-Validation Results
avg_result.append(['MNB', np.mean(mnb_acc), np.mean(mnb_pre), np.mean(mnb_rec), np.mean(mnb_f1)])
avg_result.append(['LR', np.mean(lr_acc), np.mean(lr_pre), np.mean(lr_rec), np.mean(lr_f1)])
avg_result.append(['SVM', np.mean(svm_acc), np.mean(svm_pre), np.mean(svm_rec), np.mean(svm_f1)])
avg_result.append(['RF', np.mean(rf_acc), np.mean(rf_pre), np.mean(rf_rec), np.mean(rf_f1)])
avg_result.append(['XGB', np.mean(xgb_acc), np.mean(xgb_pre), np.mean(xgb_rec), np.mean(xgb_f1)])
print(tabulate(avg_result, headers=h))

Model      Accuracy    Precision    Recall       F1
-------  ----------  -----------  --------  -------
MNB            88.4      88.2936   88.8529  88.261
LR             86        90.9406   82.8164  86.4671
SVM            86.6      89.3764   84.7021  86.7546
RF             90.4      89.7616   90.8911  90.2606
XGB            88.4      88.7007   87.931   88.2717


In [ ]:
# RF with TFIDF only
X = final_df[final_df.columns[6:-1]]
y = final_df['label']
tfidf_acc, tfidf_pre, tfidf_rec, tfidf_f1 = [], [], [], []
tfidf_result = []
for train_index, test_index in kf.split(X):
  num_fold -= 1
  Train_X, Test_X = X.iloc[train_index], X.iloc[test_index]
  Train_Y, Test_Y = y.iloc[train_index], y.iloc[test_index]

  RF = RandomForestClassifier(n_estimators=100)
  RF.fit(Train_X,Train_Y)
  predictions_RF = RF.predict(Test_X)
  rf_acc.append(accuracy_score(predictions_RF, Test_Y)*100)
  rf_pre.append(precision_score(predictions_RF, Test_Y)*100)
  rf_rec.append(recall_score(predictions_RF, Test_Y)*100)
  rf_f1.append(f1_score(predictions_RF, Test_Y)*100)

tfidf_result.append(['RF with TFIDF', np.mean(rf_acc), np.mean(rf_pre), np.mean(rf_rec), np.mean(rf_f1)])
print(tabulate(tfidf_result, headers=h))

Model            Accuracy    Precision    Recall       F1
-------------  ----------  -----------  --------  -------
RF with TFIDF          85      84.1853   85.3107  84.6138


In [ ]:
# RF with Watson NLU Features only (Affective Text Features)
X = final_df[final_df.columns[:6]]
y = final_df['label']
atf_acc, atf_pre, atf_rec, atf_f1 = [], [], [], []
atf_result = []
for train_index, test_index in kf.split(X):
  num_fold -= 1
  Train_X, Test_X = X.iloc[train_index], X.iloc[test_index]
  Train_Y, Test_Y = y.iloc[train_index], y.iloc[test_index]

  RF = RandomForestClassifier(n_estimators=100)
  RF.fit(Train_X,Train_Y)
  predictions_RF = RF.predict(Test_X)
  atf_acc.append(accuracy_score(predictions_RF, Test_Y)*100)
  atf_pre.append(precision_score(predictions_RF, Test_Y)*100)
  atf_rec.append(recall_score(predictions_RF, Test_Y)*100)
  atf_f1.append(f1_score(predictions_RF, Test_Y)*100)

atf_result.append(['RF with ATF', np.mean(atf_acc), np.mean(atf_pre), np.mean(atf_rec), np.mean(atf_f1)])
print(tabulate(atf_result, headers=h))

Model          Accuracy    Precision    Recall       F1
-----------  ----------  -----------  --------  -------
RF with ATF        86.8      86.9138    86.639  86.6712
